In [1]:
from typing import List

from daft_func import Pipeline, Runner, func
from examples.retrieval import (
    IdentityReranker,
    Query,
    RerankedHit,
    Reranker,
    RetrievalResult,
    Retriever,
    ToyRetriever,
)

In [2]:
from typing import Dict


@func(output="index_path", cache=True)
def index(retriever: Retriever, corpus: Dict[str, str], test: bool = True) -> str:
    index_path = retriever.index(corpus)
    return index_path


@func(output="hits", map_axis="query", key_attr="query_uuid", cache=True)
def retrieve(
    retriever: Retriever, query: Query, top_k: int, index_path: str
) -> RetrievalResult:
    return retriever.retrieve(index_path, query, top_k=top_k)


@func(output="reranked_hits", map_axis="query", key_attr="query_uuid", cache=True)
def rerank(
    reranker: Reranker, query: Query, hits: RetrievalResult, top_k: int
) -> List[RerankedHit]:
    return reranker.rerank(query, hits, top_k=top_k)


pipeline = Pipeline(functions=[index, retrieve, rerank])
pipeline.visualize()

In [3]:
corpus = {
    "d1": "a quick brown fox jumps",
    "d2": "brown dog sleeps",
    "d3": "five boxing wizards jump quickly",
}

single_inputs = {
    "retriever": ToyRetriever(),
    "corpus": corpus,
    "reranker": IdentityReranker(),
    "query": Query(query_uuid="q1", text="quick brown"),
    "top_k": 2,
}

In [4]:
from daft_func import CacheConfig

# Create runner with auto mode (chooses based on batch size)
runner = Runner(
    pipeline=pipeline,
    mode="local",
    batch_threshold=2,
    cache_config=CacheConfig(enabled=True),  # , backend=DiskCache(cache_dir=".cache")),
)

In [5]:
result = runner.run(inputs=single_inputs)  # should show misses


[CACHE] index_path: ✗ MISS (0.01s) | hits: ✗ MISS (0.00s) | reranked_hits: ✗ MISS (0.00s)


In [6]:
result = runner.run(inputs=single_inputs)  # should show misses


[CACHE] index_path: ✓ HIT | hits: ✓ HIT | reranked_hits: ✓ HIT


In [7]:
corpus = {
    "d1": "a quick brown fox jumps",
    "d2": "brown dog sleeps",
    "d3": "five boxing wizards jump quickly",
}

single_inputs = {
    "retriever": ToyRetriever(),
    "corpus": corpus,
    "reranker": IdentityReranker(),
    "query": Query(query_uuid="q1", text="quick brown"),
    "top_k": 2,
}

In [8]:
result = runner.run(inputs=single_inputs)  # should show hits


[CACHE] index_path: ✓ HIT | hits: ✓ HIT | reranked_hits: ✓ HIT


In [9]:
corpus = {
    "d1": "a quick brown fox jumps",
    "d2": "brown dog sleeps",
    "d3": "five boxing wizards jump quickly",
}

single_inputs = {
    "retriever": ToyRetriever(),
    "corpus": corpus,
    "reranker": IdentityReranker(),
    "query": Query(query_uuid="q1", text="quick brown"),
    "top_k": 2,
}

In [10]:
result = runner.run(
    inputs=single_inputs
)  # should show hits, but showing misses, probably because of the classes


[CACHE] index_path: ✓ HIT | hits: ✓ HIT | reranked_hits: ✓ HIT


In [11]:
multi_inputs = {
    "corpus": corpus,
    "retriever": ToyRetriever(),
    "reranker": IdentityReranker(),
    "query": [
        Query(query_uuid="q1", text="quick brown"),
        Query(query_uuid="q2", text="wizards jump"),
        Query(query_uuid="q3", text="brown dog"),
    ],
    "top_k": 2,
}

In [12]:
result = runner.run(inputs=multi_inputs)


[CACHE] index_path: ✓ HIT | hits: ✓ HIT | reranked_hits: ✓ HIT | index_path: ✓ HIT | hits: ✗ MISS (0.00s) | reranked_hits: ✗ MISS (0.00s) | index_path: ✓ HIT | hits: ✗ MISS (0.00s) | reranked_hits: ✗ MISS (0.00s)


In [16]:
result["reranked_hits"]

[[RerankedHit(query_uuid='q1', doc_id='d1', score=2.0),
  RerankedHit(query_uuid='q1', doc_id='d2', score=1.0)],
 [RerankedHit(query_uuid='q2', doc_id='d3', score=2.0),
  RerankedHit(query_uuid='q2', doc_id='d1', score=1.0)],
 [RerankedHit(query_uuid='q3', doc_id='d2', score=2.0),
  RerankedHit(query_uuid='q3', doc_id='d1', score=1.0)]]

In [17]:
for mode in ["local", "daft", "auto"]:
    runner = Runner(
        pipeline=pipeline,
        mode=mode,
        batch_threshold=2,
        cache_config=CacheConfig(enabled=True, cache_dir=".cache"),
    )
    result = runner.run(inputs=multi_inputs)
    print(
        f"✅ {mode.upper():5s} mode: {len(result['reranked_hits'])} queries processed"
    )

print("\n" + "=" * 70)
print("🎉 Demo complete!")
print("=" * 70)

TypeError: CacheConfig.__init__() got an unexpected keyword argument 'cache_dir'

In [ ]:
from time import sleep

from daft_func import func


@func(output="embeddings", cache=True, cache_key="model_v1")
def encode(text: str) -> list:
    sleep(6)
    return "s"


@func(output="result", cache=True)
def process(embeddings: list, threshold: float) -> dict:
    sleep(2)
    return "t"


pipeline = Pipeline(functions=[encode, process])
cache_config = CacheConfig(enabled=True, cache_dir=".cache")
runner = Runner(pipeline=pipeline, cache_config=cache_config)

# First run: executes both
result1 = runner.run(inputs={"text": "hello", "threshold": 0.5})


[CACHE] embeddings: ✗ MISS (6.01s) | result: ✗ MISS (2.01s)


In [ ]:
# Second run, change threshold: encode cached, process re-executes
result2 = runner.run(inputs={"text": "hello", "threshold": 0.8})

KeyboardInterrupt: 

In [ ]:
from daft_func import Pipeline, func


# Define a simple pipeline
@func(output="doubled")
def double(x: int) -> int:
    """Double the input value."""
    return x * 2


@func(output="result")
def add_value(doubled: int, offset: int = 5) -> int:
    """Add an offset to the doubled value."""
    return doubled + offset


# Create pipeline with explicit functions
pipeline = Pipeline(functions=[double, add_value])

In [ ]:
# Create and display visualization
pipeline.visualize()

In [ ]:
from pydantic import BaseModel

from daft_func import Runner, func


# 1. Define your data models
class Query(BaseModel):
    id: str
    text: str


class Result(BaseModel):
    id: str
    score: float


@func(output="results", map_axis="query", key_attr="id")
def process(query: Query, threshold: float) -> Result:
    score = len(query.text) * threshold
    return Result(id=query.id, score=score)


# 3. Create pipeline and runner
pipeline = Pipeline(functions=[process])
runner = Runner(pipeline=pipeline)

In [ ]:
outputs = runner.run(
    inputs={
        "query": [Query(id="q1", text="hello")],
        "threshold": 0.5,
    }
)

print(outputs["results"])
# [Result(id='q1', score=2.5), Result(id='q2', score=2.5)]

[Result(id='q1', score=2.5)]


In [ ]:
outputs = runner.run(
    inputs={
        "query": [
            Query(id="q1", text="hello"),
            Query(id="q2", text="world"),
        ],
        "threshold": 0.5,
    }
)

print(outputs["results"])
# [Result(id='q1', score=2.5), Result(id='q2', score=2.5)]

[Result(id='q1', score=2.5), Result(id='q2', score=2.5)]
